In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import os
from urllib.request import urlopen, quote,URLError, HTTPError
import time

In [ ]:
# 导入数据
df = pd.read_excel(r"G:\\work\\logistica\\stations\\JDStations\\yuanURL\\浙江/浙江.xlsx")
data_city = pd.read_excel(r"G:\work\logistica\stations\JDStations\stations\zhejiangcityID.xlsx")
province = "浙江省"

In [ ]:
ak_gaode='c71d9eda293d20db64955275557d92d4'

In [ ]:
# 导出数据路径
outpath = "G:\\work\\logistica\\stations\\JDStations\\stations\\website\\"
name_ANSI = "JDstations_zhejiang_gaode.xlsx"

In [ ]:
# 建立url池
urls = []
for a in df["url"]:
    urls.append("http://www.jdwl.com/site/querySiteList?" + a)

print("验证完整地址：")
print(urls[0:3])

In [ ]:
# 遍历获取JD官网网点信息
def get_JD_siteInfo(urls):
    JD_siteList = []
    for url in urls:
        request = requests.get(url)
        soup = BeautifulSoup(request.content, "lxml").get_text()
        jsonlist = json.loads(soup)
        for i in range(len(jsonlist)):
            JD_siteDict = {}
            JD_siteDict["JD_siteBusinessName"] = jsonlist[i]["siteBusinessName"]
            JD_siteDict["JD_siteProvinceId"] = jsonlist[i]["provinceId"]
            JD_siteDict["JD_siteCityId"] = jsonlist[i]["cityId"]
            JD_siteDict["JD_siteCountyId"] = jsonlist[i]["countyId"]
            JD_siteDict["siteName"] = jsonlist[i]["siteName"]
            if 'address' in jsonlist[i]:
                JD_siteDict["JD_siteAddress"] = jsonlist[i]["address"]
            else:
                pass
            if 'latitude' in jsonlist[i]:
                JD_siteDict["JD_siteLatitude"] = jsonlist[i]['latitude']
            else:
                pass  
            if 'longitude' in jsonlist[i]:
                JD_siteDict["JD_siteLongitude"] = jsonlist[i]['longitude']
            else:
                pass       
            if "telephone" in jsonlist[i]:
                JD_siteDict["JD_siteTelephone"] = jsonlist[i]["telephone"]
            else:
                pass
            JD_siteList.append(JD_siteDict)
    return JD_siteList

In [ ]:
def get_latlng_Gaode(address,ak):
    ak = ak_gaode
    url="http://restapi.amap.com/v3/geocode/geo?key=%s&address=%s"%(ak,address)
    data=requests.get(url)
    contest=data.json()
    return contest

In [ ]:
def get_JD_siteInfo_gaode(urls):
    start_time = time.time()
    JD_siteList_gaode = []
    for b in urls:
        # print(b)
        JD_siteDict_gaode = {}
        try:
            temp=get_latlng_Gaode(b)
        except HTTPError as e:
            print("请求出错")
            pass
        else:    
            if ('geocodes' in temp):
                JD_siteDict_gaode["JD_siteLocation_gaode"] = temp['geocodes'][0]['location']
                JD_siteDict_gaode["JD_siteFormatted_address_gaode"] = temp['geocodes'][0]['formatted_address']
                JD_siteDict_gaode["JD_siteProvince_gaode"] = temp['geocodes'][0]['province']
                JD_siteDict_gaode["JD_siteCountry_gaode"] = temp['geocodes'][0]['country']
                if ('citycode' in temp):
                    JD_siteDict_gaode["JD_siteCitycode_gaode"] = temp['geocodes'][0]['citycode']
                else:
                    pass
                JD_siteDict_gaode["JD_siteCity_gaode"] = temp['geocodes'][0]['city']
                JD_siteDict_gaode["JD_siteDistrict_gaode"] = temp['geocodes'][0]['district']
                JD_siteDict_gaode["JD_siteLevel_gaode"] = temp['geocodes'][0]['level']
                JD_siteDict_gaode["JD_siteTownship_gaode"] = temp['geocodes'][0]['township']
                JD_siteDict_gaode["JD_siteAdcode_gaode"] = temp['geocodes'][0]['adcode']
                JD_siteDict_gaode["JD_siteStreet_gaode"] = temp['geocodes'][0]['street']
                JD_siteDict_gaode["JD_siteNumber_gaode"] = temp['geocodes'][0]['number']
            else:
                pass
        JD_siteList_gaode.append(JD_siteDict_gaode)
    #     time.sleep(1)
    end_time = time.time()
    print("总用时：",end_time-start_time)
    return JD_siteList_gaode

In [ ]:
# df化
df_JD_siteInfo = pd.DataFrame(get_JD_siteInfo(urls))

In [ ]:
# 去除空白地址所在行
print(df_JD_siteInfo['JD_siteAddress'].isnull().value_counts())
df_JD_siteInfo = df_JD_siteInfo.dropna(subset=["JD_siteAddress"])
print(df_JD_siteInfo['JD_siteAddress'].isnull().value_counts())

In [ ]:
print(df_JD_siteInfo.columns)
print("*"*30)
print(data_city.columns)

In [ ]:
df["JD_siteAddress_all"] = province + df['cityname'] + df['JD_siteAddress']

In [ ]:
print("验证完整地址：")
print(df["siteAddress_all"][0:3])

In [ ]:
# df化
df_JD_siteList_gaode = pd.DataFrame(get_JD_siteInfo(urls))

In [ ]:
# 合并并清洗
df_JD_siteInfo_gaode = pd.concat([df_JD_siteList_gaode,df_JD_siteInfo],axis=1)
df_JD_siteInfo_gaode = df_JD_siteInfo_gaode.drop(columns=['JD_siteLongitude','JD_siteLatitude'])
split_gaode = pd.DataFrame((x.split(',') for x in df_JD_siteInfo_gaode['JD_siteLocation_gaode']), index=df_JD_siteInfo_gaode.index, columns=['JD_siteLng_gaode','JD_siteLat_gaode'])
df_JD_siteInfo_gaode = pd.merge(df_JD_siteInfo_gaode,split_gaode,left_index=True,right_index=True)

In [ ]:
# 最后检查
print("JD官网网点信息：" + str(df_JD_siteInfo.shape))
print(df.shape)
print(df_A.shape)
print(df_all.shape)
print(df_allA.shape)